Importacion de librerias

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

from sklearn.model_selection import train_test_split, cross_val_score, learning_curve
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import (mean_squared_error, mean_absolute_error, 
                             r2_score, mean_absolute_percentage_error)

plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")
plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['font.size'] = 10

print("=" * 80)
print("MODELO DE REGRESIÓN LINEAL - PREDICCIÓN DE PRECIOS DE AGUACATE")
print("=" * 80)
print(f"Fecha de ejecución: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print("=" * 80)


Carga y exploración inicial de datos

In [ ]:
print("\n[1] CARGANDO DATOS PREPROCESADOS...")
print("-" * 80)

df = pd.read_csv('../../data/processed/avocado_clean.csv')

print(f"✓ Datos cargados exitosamente")
print(f"  - Dimensiones: {df.shape[0]} filas × {df.shape[1]} columnas")
print(f"  - Memoria utilizada: {df.memory_usage(deep=True).sum() / 1024**2:.2f} MB")

print("\n[2] INFORMACIÓN DEL DATASET")
print("-" * 80)
print(df.info())

print("\n[3] ESTADÍSTICAS DESCRIPTIVAS")
print("-" * 80)
print(df.describe())

Preparación de datos para regresion lineal

In [ ]:
print("\n[4] PREPARACIÓN DE VARIABLES")
print("-" * 80)

target = 'AveragePrice'
print(f"Variable objetivo: {target}")

features_to_exclude = [target, 'Date'] if 'Date' in df.columns else [target]
feature_columns = [col for col in df.columns if col not in features_to_exclude]

print(f"\nFeatures seleccionadas ({len(feature_columns)}):")
for i, col in enumerate(feature_columns, 1):
    print(f"  {i}. {col}")

X = df[feature_columns]
y = df[target]

print(f"\n✓ Datos preparados:")
print(f"  - X shape: {X.shape}")
print(f"  - y shape: {y.shape}")

Análisis de correlación

In [ ]:
print("\n[5] ANÁLISIS DE CORRELACIÓN")
print("-" * 80)

correlation = df[feature_columns + [target]].corr()[target].sort_values(ascending=False)
print("\nTop 10 variables más correlacionadas con AveragePrice:")
print(correlation.head(11)[1:])  # Excluir la correlación consigo misma

plt.figure(figsize=(14, 10))
correlation_matrix = df[feature_columns + [target]].corr()
mask = np.triu(np.ones_like(correlation_matrix, dtype=bool))
sns.heatmap(correlation_matrix, mask=mask, annot=True, fmt='.2f', 
            cmap='coolwarm', center=0, square=True, linewidths=1,
            cbar_kws={"shrink": 0.8})
plt.title('Matriz de Correlación - Features vs AveragePrice', 
          fontsize=16, fontweight='bold', pad=20)
plt.tight_layout()
plt.savefig('../results/linear_regression_correlation_matrix.png', dpi=300, bbox_inches='tight')
print("\n✓ Matriz de correlación guardada")
plt.show()

División de datos (train/test)

In [ ]:
print("\n[6] DIVISIÓN DE DATOS")
print("-" * 80)

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

print(f"✓ División completada:")
print(f"  - Conjunto de entrenamiento: {X_train.shape[0]} muestras ({X_train.shape[0]/len(X)*100:.1f}%)")
print(f"  - Conjunto de prueba: {X_test.shape[0]} muestras ({X_test.shape[0]/len(X)*100:.1f}%)")
print(f"\n  Estadísticas de la variable objetivo:")
print(f"  - Train: Media={y_train.mean():.3f}, Std={y_train.std():.3f}")
print(f"  - Test:  Media={y_test.mean():.3f}, Std={y_test.std():.3f}")


Normalizción de datos

In [ ]:
print("\n[7] NORMALIZACIÓN DE FEATURES")
print("-" * 80)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

print("✓ Normalización aplicada (StandardScaler)")
print(f"  - Media features normalizadas: {X_train_scaled.mean():.2e}")
print(f"  - Desviación estándar: {X_train_scaled.std():.2e}")


Entrenamiento de modelo

In [ ]:
print("\n[8] ENTRENAMIENTO DEL MODELO DE REGRESIÓN LINEAL")
print("-" * 80)

model = LinearRegression()
print("Entrenando modelo...")
model.fit(X_train_scaled, y_train)
print("✓ Modelo entrenado exitosamente")

print(f"\n  Parámetros del modelo:")
print(f"  - Número de coeficientes: {len(model.coef_)}")
print(f"  - Intercepto: {model.intercept_:.4f}")


Predicciones

In [ ]:
print("\n[9] GENERANDO PREDICCIONES")
print("-" * 80)

y_train_pred = model.predict(X_train_scaled)
y_test_pred = model.predict(X_test_scaled)

print("✓ Predicciones generadas")
print(f"  - Predicciones train: {len(y_train_pred)}")
print(f"  - Predicciones test: {len(y_test_pred)}")


Evaluación del modelo

In [ ]:
print("\n[10] EVALUACIÓN DEL MODELO")
print("=" * 80)

def calculate_metrics(y_true, y_pred, dataset_name):
    """Calcula y muestra métricas de evaluación"""
    mse = mean_squared_error(y_true, y_pred)
    rmse = np.sqrt(mse)
    mae = mean_absolute_error(y_true, y_pred)
    r2 = r2_score(y_true, y_pred)
    mape = mean_absolute_percentage_error(y_true, y_pred) * 100
    
    print(f"\n{dataset_name}:")
    print(f"  - R² Score:                {r2:.6f}")
    print(f"  - Mean Squared Error (MSE): {mse:.6f}")
    print(f"  - Root Mean Squared Error:  {rmse:.6f}")
    print(f"  - Mean Absolute Error:      {mae:.6f}")
    print(f"  - MAPE:                     {mape:.4f}%")
    
    return {'MSE': mse, 'RMSE': rmse, 'MAE': mae, 'R2': r2, 'MAPE': mape}

train_metrics = calculate_metrics(y_train, y_train_pred, "CONJUNTO DE ENTRENAMIENTO")
test_metrics = calculate_metrics(y_test, y_test_pred, "CONJUNTO DE PRUEBA")

print("\n[11] VALIDACIÓN CRUZADA (K-FOLD = 5)")
print("-" * 80)
cv_scores = cross_val_score(model, X_train_scaled, y_train, cv=5, 
                            scoring='r2', n_jobs=-1)
print(f"✓ R² Scores por fold: {cv_scores}")
print(f"  - Media: {cv_scores.mean():.6f}")
print(f"  - Desviación estándar: {cv_scores.std():.6f}")


Análisis de coeficientes

In [ ]:
print("\n[12] ANÁLISIS DE COEFICIENTES (IMPORTANCIA DE FEATURES)")
print("=" * 80)

coefficients = pd.DataFrame({
    'Feature': feature_columns,
    'Coefficient': model.coef_,
    'Abs_Coefficient': np.abs(model.coef_)
}).sort_values('Abs_Coefficient', ascending=False)

print("\nTop 10 Features más importantes:")
print(coefficients.head(10).to_string(index=False))

plt.figure(figsize=(12, 8))
top_n = 15
top_coef = coefficients.head(top_n)
colors = ['green' if x > 0 else 'red' for x in top_coef['Coefficient']]
plt.barh(range(len(top_coef)), top_coef['Coefficient'], color=colors, alpha=0.7)
plt.yticks(range(len(top_coef)), top_coef['Feature'])
plt.xlabel('Coeficiente', fontsize=12, fontweight='bold')
plt.title(f'Top {top_n} Features más Importantes (Regresión Lineal)', 
          fontsize=14, fontweight='bold', pad=15)
plt.axvline(x=0, color='black', linestyle='--', linewidth=1)
plt.grid(axis='x', alpha=0.3)
plt.tight_layout()
plt.savefig('../results/linear_regression_feature_importance.png', dpi=300, bbox_inches='tight')
print("\n✓ Gráfico de importancia guardado")
plt.show()

Visualización de resultados

In [ ]:
print("\n[13] GENERANDO VISUALIZACIONES")
print("-" * 80)

fig, axes = plt.subplots(2, 2, figsize=(16, 12))
fig.suptitle('Análisis de Resultados - Regresión Lineal', 
             fontsize=16, fontweight='bold', y=1.00)

# 1. Valores Reales vs Predicciones (Test)
ax1 = axes[0, 0]
ax1.scatter(y_test, y_test_pred, alpha=0.5, s=30, edgecolors='black', linewidth=0.5)
ax1.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 
         'r--', lw=2, label='Predicción Perfecta')
ax1.set_xlabel('Valores Reales', fontsize=11, fontweight='bold')
ax1.set_ylabel('Valores Predichos', fontsize=11, fontweight='bold')
ax1.set_title(f'Predicciones vs Valores Reales (Test)\nR² = {test_metrics["R2"]:.4f}', 
              fontsize=12, fontweight='bold')
ax1.legend()
ax1.grid(True, alpha=0.3)

# 2. Residuos
ax2 = axes[0, 1]
residuals = y_test - y_test_pred
ax2.scatter(y_test_pred, residuals, alpha=0.5, s=30, edgecolors='black', linewidth=0.5)
ax2.axhline(y=0, color='r', linestyle='--', linewidth=2)
ax2.set_xlabel('Valores Predichos', fontsize=11, fontweight='bold')
ax2.set_ylabel('Residuos', fontsize=11, fontweight='bold')
ax2.set_title('Análisis de Residuos\n(Test Set)', fontsize=12, fontweight='bold')
ax2.grid(True, alpha=0.3)

# 3. Distribución de Residuos
ax3 = axes[1, 0]
ax3.hist(residuals, bins=50, edgecolor='black', alpha=0.7)
ax3.axvline(x=0, color='r', linestyle='--', linewidth=2)
ax3.set_xlabel('Residuos', fontsize=11, fontweight='bold')
ax3.set_ylabel('Frecuencia', fontsize=11, fontweight='bold')
ax3.set_title(f'Distribución de Residuos\nMedia = {residuals.mean():.4f}, Std = {residuals.std():.4f}', 
              fontsize=12, fontweight='bold')
ax3.grid(True, alpha=0.3, axis='y')

# 4. Q-Q Plot
ax4 = axes[1, 1]
from scipy import stats
stats.probplot(residuals, dist="norm", plot=ax4)
ax4.set_title('Q-Q Plot (Normalidad de Residuos)', fontsize=12, fontweight='bold')
ax4.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('../results/linear_regression_analysis.png', dpi=300, bbox_inches='tight')
print("✓ Visualizaciones guardadas")
plt.show()

Curva de aprendizaje

In [ ]:
print("\n[14] CURVA DE APRENDIZAJE")
print("-" * 80)

train_sizes, train_scores, val_scores = learning_curve(
    model, X_train_scaled, y_train, cv=5, n_jobs=-1,
    train_sizes=np.linspace(0.1, 1.0, 10), scoring='r2'
)

train_mean = np.mean(train_scores, axis=1)
train_std = np.std(train_scores, axis=1)
val_mean = np.mean(val_scores, axis=1)
val_std = np.std(val_scores, axis=1)

plt.figure(figsize=(10, 6))
plt.plot(train_sizes, train_mean, label='Training Score', color='blue', marker='o')
plt.fill_between(train_sizes, train_mean - train_std, train_mean + train_std, alpha=0.15, color='blue')
plt.plot(train_sizes, val_mean, label='Validation Score', color='green', marker='s')
plt.fill_between(train_sizes, val_mean - val_std, val_mean + val_std, alpha=0.15, color='green')
plt.xlabel('Tamaño del Conjunto de Entrenamiento', fontsize=12, fontweight='bold')
plt.ylabel('R² Score', fontsize=12, fontweight='bold')
plt.title('Curva de Aprendizaje - Regresión Lineal', fontsize=14, fontweight='bold', pad=15)
plt.legend(loc='best')
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.savefig('../results/linear_regression_learning_curve.png', dpi=300, bbox_inches='tight')
print("✓ Curva de aprendizaje guardada")
plt.show()